# Star-Wars API playground
A first test to use the star wars API with pandas to create a csv

In [1]:
import numpy as np
import pandas as pd
import requests
import json

url = 'https://swapi.dev/api/'
JSONContent = requests.get(url).json()
content = json.dumps(JSONContent, indent = 4, sort_keys=True)

In [2]:
all_people_list = []
# For each character, we access its information through its API.
# I knew the number from a first request I have not documented here.
# There must be a better way.

for x in range(1, 83):
    JSONContent = requests.get('https://swapi.dev/api/people/' + str(x)).json()
    all_people_list.append(JSONContent)                 
people_df = pd.DataFrame(all_people_list)
people_df.tail()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url,detail
77,Shaak Ti,178,57,none,"red, blue, white",black,unknown,female,http://swapi.dev/api/planets/58/,"[http://swapi.dev/api/films/5/, http://swapi.d...",[http://swapi.dev/api/species/35/],[],[],2014-12-20T18:44:01.103000Z,2014-12-20T21:17:50.486000Z,http://swapi.dev/api/people/78/,NaN
78,Grievous,216,159,none,"brown, white","green, yellow",unknown,male,http://swapi.dev/api/planets/59/,[http://swapi.dev/api/films/6/],[http://swapi.dev/api/species/36/],[http://swapi.dev/api/vehicles/60/],[http://swapi.dev/api/starships/74/],2014-12-20T19:43:53.348000Z,2014-12-20T21:17:50.488000Z,http://swapi.dev/api/people/79/,NaN
79,Tarfful,234,136,brown,brown,blue,unknown,male,http://swapi.dev/api/planets/14/,[http://swapi.dev/api/films/6/],[http://swapi.dev/api/species/3/],[],[],2014-12-20T19:46:34.209000Z,2014-12-20T21:17:50.491000Z,http://swapi.dev/api/people/80/,NaN
80,Raymus Antilles,188,79,brown,light,brown,unknown,male,http://swapi.dev/api/planets/2/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[],[],[],2014-12-20T19:49:35.583000Z,2014-12-20T21:17:50.493000Z,http://swapi.dev/api/people/81/,NaN
81,Sly Moore,178,48,none,pale,white,unknown,female,http://swapi.dev/api/planets/60/,"[http://swapi.dev/api/films/5/, http://swapi.d...",[],[],[],2014-12-20T20:18:37.619000Z,2014-12-20T21:17:50.496000Z,http://swapi.dev/api/people/82/,NaN


First check to see interesting columns and trying out to convert to csv.

In [3]:
dataset_to_csv = people_df.loc[:, ['name', 'gender', 'height', 'eye_color']].copy()

# dataset_to_csv.to_csv('star_wars_characters.csv', index=False)

In [6]:
# A list where I store the values from the api that I request below
def get_all_planets_data():
    # First API request and definition of JSONContent variable
    planets_url = 'https://swapi.dev/api/planets/'
    JSONContent = requests.get(planets_url).json()
    all_planets_list = []
    
    all_planets_list.extend(JSONContent['results'])
    # Looping through the paginated API and storing it's results to all_planets_list
    while JSONContent['next']:
        JSONContent = requests.get(JSONContent['next']).json()
        all_planets_list.extend(JSONContent['results'])
    # converting the list into a pd.df
    df = pd.DataFrame(all_planets_list)
    return df

planets_df = get_all_planets_data()

# Iterate over the list (if globally defined) including an index using the built-in enumerate function
# for i, planet in enumerate(all_planets_list):
#    print(i, planet['name'], planet['residents'], planet['population'], planet['url'])

0      http://swapi.dev/api/planets/1/
1      http://swapi.dev/api/planets/2/
2      http://swapi.dev/api/planets/3/
3      http://swapi.dev/api/planets/4/
4      http://swapi.dev/api/planets/5/
5      http://swapi.dev/api/planets/6/
6      http://swapi.dev/api/planets/7/
7      http://swapi.dev/api/planets/8/
8      http://swapi.dev/api/planets/9/
9     http://swapi.dev/api/planets/10/
10    http://swapi.dev/api/planets/11/
11    http://swapi.dev/api/planets/12/
12    http://swapi.dev/api/planets/13/
13    http://swapi.dev/api/planets/14/
14    http://swapi.dev/api/planets/15/
15    http://swapi.dev/api/planets/16/
16    http://swapi.dev/api/planets/17/
17    http://swapi.dev/api/planets/18/
18    http://swapi.dev/api/planets/19/
19    http://swapi.dev/api/planets/20/
20    http://swapi.dev/api/planets/21/
21    http://swapi.dev/api/planets/22/
22    http://swapi.dev/api/planets/23/
23    http://swapi.dev/api/planets/24/
24    http://swapi.dev/api/planets/25/
25    http://swapi.dev/ap

## Merge the people and planet DataFrames
The merge needs to happen on a unique identifier people_df['homeworld'] and planet_df['url']. Afterwards I can rename columns to my wishes and create a new df with only the columns I want to convert to a CSV file.

In [10]:
merged_df = people_df.merge(
    planets_df,
    how='left',
    left_on=['homeworld'],
    right_on=['url'],
    suffixes=('', '_planet')
)

In [11]:
merged_df.columns

Index(['name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color',
       'birth_year', 'gender', 'homeworld', 'films', 'species', 'vehicles',
       'starships', 'created', 'edited', 'url', 'detail', 'name_planet',
       'rotation_period', 'orbital_period', 'diameter', 'climate', 'gravity',
       'terrain', 'surface_water', 'population', 'residents', 'films_planet',
       'created_planet', 'edited_planet', 'url_planet'],
      dtype='object')

In [14]:
final_df = merged_df.rename(columns={
    "name_planet": "home",
}, errors="raise")

header = ["InviteTime (Oracle)", "Orig Number", "Orig IP Address", "Dest Number"]
df.to_csv('output.csv', columns = header)
final_df.loc[['name', 'gender', 'home', 'population']]

,name,gender,home,population
0,Luke Skywalker,male,Tatooine,200000
1,C-3PO,n/a,Tatooine,200000
2,R2-D2,n/a,Naboo,4500000000
3,Darth Vader,male,Tatooine,200000
4,Leia Organa,female,Alderaan,2000000000
...,...,...,...,...
77,Shaak Ti,female,Shili,unknown
78,Grievous,male,Kalee,4000000000
79,Tarfful,male,Kashyyyk,45000000
80,Raymus Antilles,male,Alderaan,2000000000


In [16]:
final_df.to_csv('star_wars_characters.csv2.csv', index=False)